# 5. Inferencia y grados de libertad

Importamos los datos y módulos a usar.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle  

import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '../Data/'

X = np.load(path + 'X.dat', allow_pickle= True)
B = np.load(path + 'B.dat', allow_pickle= True)
Y = np.load(path + 'Y.dat', allow_pickle= True)

In [4]:
result =  sm.load(path + 'ols_results.pickle')

In [5]:
with open("../Data/regressor_columns.pickle", "rb") as fp:   #Pickling
...   regressor_columns = pickle.load(fp)

In [6]:
df = pd.read_csv(path+'hour-processed2.csv', delimiter=',')

In [7]:
regressor_columns

['temp',
 'hum',
 'windspeed',
 'yr',
 'hr_sin',
 'hr_cos',
 'weekday_sin',
 'weekday_cos',
 'day_sin',
 'day_cos',
 'mnth_sin',
 'holiday',
 'rush_hour',
 'weekend',
 'season_2',
 'season_3',
 'season_4',
 'weathersit_2',
 'weathersit_3',
 'weathersit_4']

## 5.1 Calculo de grados de libertad

Los grados de libertad de modelo corresponden a número de coeficientes B estimados, asociados a variables regresoras. Es decir, es calculado como:

$$Df_{Model} = m - 1 = k$$

In [8]:
Df_Model = len(B) - 1

Df_Model

20

In [9]:
result.df_model

20.0

Por su parte, los grados de libertad de los residuos corresponden al número de observaciones n menos el total de coeficientes estimados:

$$Df_{Residuals} = n - m = n - (k + 1)$$

In [10]:
Df_Residuals = len(Y) - len(B)

Df_Residuals

17358

In [11]:
result.df_resid

np.float64(17358.0)

Comparando con los grados de libertad total, que se calcula como n - 1, por la perdida de un grado de libertad por la estimación de $\overline{Y}$:

$$Df_{Total} = n - 1$$

Por lo que se cumple que:

$$Df_{Total} = n - 1 - k + k$$
$$Df_{Total} = n - (k + 1) + k$$
$$Df_{Total} = Df_{Residuals} + Df_{Model}$$

In [12]:
Df_Total = len(Y) - 1

Df_Total

17378

In [13]:
Df_Residuals + Df_Model

17378

## 5.2 Calculo de error estandar

Para calcular el error estándar de cada coeficiente, primero calculamos el error medio cuadrático, usando la expresión:

$$MSE = \sum \frac{(\hat{y} - y)^2}{df_{residuals}}$$

In [14]:
Y_pred = X @ B

MSR = np.sum((Y_pred - Y)**2)/(Df_Residuals)

MSR

np.float64(11703.523205548296)

Luego, se obtiene la matriz de covarianza de los estimadores calculada como:

$$COV = (X^T X)$$

De esta matriz, nos interesa los valores que están en la diagonal central, los cuales corresponderán a la covarianza de los estimadores, incluyendo $B_0$.

Además, dado que para calcular el error estandar es necesario dividir el error cuadratico medio sobre los valores de la covarianza, es necesario sacar su inversa. De esta manera calculamos el error estandar como:

In [15]:
cov_beta = MSR * np.linalg.inv(X.T @ X)

std_error = np.sqrt(np.diag(cov_beta))

std_error
    

array([ 2.90293353,  1.48780269,  1.11501   ,  0.88498001,  0.82835909,
        0.95294646,  0.9252434 ,  0.95581636,  1.32903021,  0.82474946,
        0.82101166,  1.81379079,  5.07064518,  1.93937058,  3.13105939,
        2.98948671,  4.67889784,  3.84119189,  2.03477839,  3.42510345,
       62.54072142])

Adicionalmente, podemos calcular errores estándar corregidos por autocorrelación en las variables. Para esto usamos el método de HAC / Newey-West, el cual se encuentra nativamente en statsmodel.

In [16]:
# Errores estándar HAC (Newey-West)
# lag = número de rezagos de autocorrelación a corregir
nw_lags = 24  # útil para datos horarios

nw_results = result.get_robustcov_results(cov_type='HAC', maxlags=nw_lags)

In [17]:
nw_results.bse

array([ 4.40934201,  2.67310381,  1.72235872,  1.2836424 ,  1.22699938,
        1.56653621,  1.49490571,  1.35032114,  1.84875224,  1.27624315,
        1.12878744,  2.97087938,  9.70859983,  5.72768631,  4.61972938,
        4.44306054,  7.06469307,  5.69749609,  2.53477038,  5.3821006 ,
       51.14970563])

Notamos que en la mayoría que los casos, el error estándar es mayor, con excepción de la última variable regresora, la cual disminuye su error estándar.

In [18]:
comparativa_bse = pd.DataFrame({
        'Variable Regresora': ['const'] + regressor_columns,
        'Sin corregir':std_error,
        'Corregido': nw_results.bse
    })

comparativa_bse

,Variable Regresora,Sin corregir,Corregido
0,const,2.902934,4.409342
1,temp,1.487803,2.673104
2,hum,1.115010,1.722359
3,windspeed,0.884980,1.283642
4,yr,0.828359,1.226999
5,hr_sin,0.952946,1.566536
6,hr_cos,0.925243,1.494906
7,weekday_sin,0.955816,1.350321
8,weekday_cos,1.329030,1.848752
9,day_sin,0.824749,1.276243


## 5.3 Testeo de hipótesis y p-valor

Con el error estándar podemos obtener un estadístico t, que nos ayude a validar la hipotesis de significancia del modelo, donde:

$$H_0: B_i = 0, \forall i > 0$</p> <p style="text-align:center;">$H_1: B_i \ne 0, \forall i > 0$$

Se entiende que si no se puede rechazar la hipotesis nula que el coeficiente asociado a un atributo $X_i$ es igual a 0, entonces ese atributo no tiene una significancia verdadera en el modelo, por lo cual puede ser descartado. El estadistico t asociado será igual a:
$$t_i = \frac{B_i}{se_i}, \forall i > 0$$

Y el p valor será igual a:
$$p_i = 2 * P(t>|t_i|), \forall i > 0$$

Una hipótesis similar se puede realizar para el intercepto $B_0$, donde se evalúa la hipótesis de que el modelo pase por el origen, es decir, $Y=0$ para $X = 0$.

## 5.4 Tabla de coeficientes

Obtenemos la tabla de coeficientes y errores estándar corregido otorgada por statmodels. Comparamos los errores estándares con los calculados:

In [20]:
nw_results.summary(xname = ['const'] + regressor_columns)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.645
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     336.0
Date:                Sun, 30 Nov 2025   Prob (F-statistic):               0.00
Time:                        15:04:49   Log-Likelihood:            -1.0605e+05
No. Observations:               17379   AIC:                         2.121e+05
Df Residuals:                   17358   BIC:                         2.123e+05
Df Model:                          20                                         
Covariance Type:                  HAC                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          130.9370      4.409     29.695      0.000     122.294     139.580
temp            44.3765      2.673     16.601      0.000      39.137      49.616
hum            -12.2448      1.722     -7.109      0.000     -15.621      -8.869
windspeed       -3.5600      1.284     -2.773      0.006      -6.076      -1.044
yr              42.9048      1.227     34.967      0.000      40.500      45.310
hr_sin         -59.3455      1.567    -37.883      0.000     -62.416     -56.275
hr_cos         -52.6864      1.495    -35.244      0.000     -55.617     -49.756
weekday_sin     -3.1659      1.350     -2.345      0.019      -5.813      -0.519
weekday_cos     -1.3072      1.849     -0.707      0.480      -4.931       2.317
day_sin          1.9161      1.276      1.501      0.133      -0.585       4.418
day_cos         -3.7188      1.129     -3.294      0.001      -5.931      -1.506
mnth_sin        -7.4187      2.971     -2.497      0.013     -13.242      -1.596
holiday        -29.0991      9.709     -2.997      0.003     -48.129     -10.069
rush_hour      159.2184      5.728     27.798      0.000     147.992     170.445
weekend         -5.3649      4.620     -1.161      0.246     -14.420       3.690
season_2        45.2242      4.443     10.179      0.000      36.515      53.933
season_3        18.8558      7.065      2.669      0.008       5.008      32.703
season_4        54.2928      5.697      9.529      0.000      43.125      65.460
weathersit_2   -12.0554      2.535     -4.756      0.000     -17.024      -7.087
weathersit_3   -67.4484      5.382    -12.532      0.000     -77.998     -56.899
weathersit_4   -59.8011     51.150     -1.169      0.242    -160.060      40.457
==============================================================================
Omnibus:                     2144.704   Durbin-Watson:                   0.729
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3997.591
Skew:                           0.807   Prob(JB):                         0.00
Kurtosis:                       4.708   Cond. No.                         101.
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 24 lags and without small sample correction
"""

Analizando la tabla, llegamos a las siguientes conclusiones:
* El intercepto obtenido es positivo, se rechaza hipotesis nula de que sea 0.
  
* Los parámetros $B_1, B_4, B_{13}, B_{15}, B_{16}$ y $B_{17}$ son positivos, con p-valor inferior a 5%, por lo que se rechaza la hipótesis nula que sean 0 y no tengan significancia en el modelo. Que sean positivos implica un incremento en el atributo $X_i$ implica un aumento en el número de bicicletas alquiladas.
  
* Los parámetros $B_2, B_3, B_5, B_6, B_7, B_{10}, B_{11}, B_{12}, B_{18}$ y $B_{19}$ son negativos, con p-valor inferior a 5%, por lo que se rechaza la hipótesis nula que sean 0 y no tengan significancia en el modelo. Que sean negativos implica un incremento en el atributo $X_i$ implica una reducción en el número de bicicletas alquiladas.
  
* Los parámetros $B_8, B_9, B_{14}$ y $B_{20}$ tienen un p-valor superior al 5%, por lo que no se puede rechazar la hipótesis nula de que es igual a 0. Por tanto, podemos asumir que su significancia en el modelo es nula, en comparación a los otros parámetros.

## 5.5 Analisis de variables significativas

De esta manera, llegamos a la conclusión que las variables más significativas del modelo son:


In [21]:
np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:20,.4f}'.format

pvalues_df = pd.DataFrame({
        'Variable Regresora': ['const'] + regressor_columns,
        'Coeficiente': nw_results.params,
        'p-value': nw_results.pvalues
    })

pvalues_df

,Variable Regresora,Coeficiente,p-value
0,const,130.9370,0.0000
1,temp,44.3765,0.0000
2,hum,-12.2448,0.0000
3,windspeed,-3.5600,0.0056
4,yr,42.9048,0.0000
5,hr_sin,-59.3455,0.0000
6,hr_cos,-52.6864,0.0000
7,weekday_sin,-3.1659,0.0191
8,weekday_cos,-1.3072,0.4795
9,day_sin,1.9161,0.1333


Teniendo en cuenta nuestros p-valores, concluimos que nuestras variables significativas son las siguientes:

In [22]:
regressor_columns_def = [regressor for regressor, pvalue in zip(['const'] + regressor_columns, nw_results.pvalues) if pvalue < 0.05 and regressor != 'const']

regressor_columns_def

['temp',
 'hum',
 'windspeed',
 'yr',
 'hr_sin',
 'hr_cos',
 'weekday_sin',
 'day_cos',
 'mnth_sin',
 'holiday',
 'rush_hour',
 'season_2',
 'season_3',
 'season_4',
 'weathersit_2',
 'weathersit_3']

En cambio las no significativas son:

In [23]:
omitted_regressors = [regressor for regressor in regressor_columns if regressor not in regressor_columns_def]

omitted_regressors

['weekday_cos', 'day_sin', 'weekend', 'weathersit_4']

### Variables Significantes

A continuación, hacemos un análisis de las variables que encontramos son significantes para el modelo

| Variable         | Tipo                     | Qué representa                           | Relevancia en la predicción             | Interpretación práctica                                                              |
| ---------------- | ------------------------ | ---------------------------------------- | --------------------------------------- | ------------------------------------------------------------------------------------ |
| **temp**         | Continua                 | Temperatura normalizada                  | **Muy alta**                            | La demanda aumenta en temperaturas cálidas; es uno de los predictores más fuertes.   |
| **hum**          | Continua                 | Humedad relativa                         | **Alta (negativa)**                     | Alta humedad reduce la demanda; anticipa lluvia e incomodidad.                       |
| **windspeed**    | Continua                 | Velocidad del viento normalizada         | **Baja a moderada**                     | Cambios leves no afectan mucho; vientos fuertes reducen un poco el uso.              |
| **yr**           | Binaria (0=2011, 1=2012) | Tendencia anual                          | **Alta**                                | 2012 tiene mucha mayor demanda que 2011 → crecimiento del sistema.                   |
| **hr_sin**       | Continua (cíclica)       | Codificación senoidal de la hora         | **Muy alta**                            | Captura patrones horarios: picos 8 AM y 5–6 PM.                                      |
| **hr_cos**       | Continua (cíclica)       | Complemento coseno de la hora            | **Muy alta**                            | Completa el ciclo horario; mejora predicción de horas tempranas/tardías.             |
| **weekday_sin**  | Continua (cíclica)       | Ciclo semanal                            | **Moderada**                            | Mayor demanda jueves–sábado; menor los lunes.                                        |
| **day_cos**      | Continua (cíclica)       | Representación alterna del ciclo semanal | **Moderada**                            | Ayuda a capturar patrones suaves entre días consecutivos.                            |
| **mnth_sin**     | Continua (cíclica)       | Estacionalidad mensual                   | **Alta**                                | Mayor uso en verano; menor en invierno.                                              |
| **holiday**      | Binaria                  | 1 si es festivo                          | **Alta**                     | La demanda cambia en festivos; suele ser menor que en fines de semana.               |
| **rush_hour**    | Binaria                  | 1 si hora pico (7–9 AM, 4–6 PM)          | **Muy alta (especialmente Registered)** | Captura el efecto laboral; permite diferenciar commuting vs recreativo.              |
| **season_2**     | Binaria                  | Primavera                                | **Alta**                            | Uso mayor que en invierno, pero menor que verano.                                    |
| **season_3**     | Binaria                  | Verano                                   | **Alta**                                | Estación con mayor demanda del año.                                                  |
| **season_4**     | Binaria                  | Otoño                                    | **Alta**                            | Demanda buena pero menos que verano.                                                 |
| **weathersit_2** | Binaria                  | Clima “nublado/mist”                     | **Moderada**                            | Reduce un poco el uso, pero no tanto como lluvia.                                    |
| **weathersit_3** | Binaria                  | Lluvia ligera / nieve ligera             | **Muy alta (negativa)**                 | Causa una caída fuerte en el uso; es de los predictores más importantes en negativo. |


---





De esta manera, calculamos nuevamente nuestro modelo, usando únicamente las variables significantes:

In [24]:
X = df[regressor_columns_def].to_numpy()

## inserción de 1
X_1 = np.insert(X, obj=0, values=1, axis=1)

X_1

array([[ 1.        , -1.33464759,  0.94776791, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -1.4385164 ,  0.89590169, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -1.4385164 ,  0.89590169, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.        , -1.23077877, -0.14142266, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -1.23077877, -0.34888753, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.        , -1.23077877,  0.11790843, ...,  0.        ,
         0.        ,  0.        ]], shape=(17379, 17))

In [25]:
X_1.shape

(17379, 17)

In [26]:
# Se recalcula el modelo con OLS
model = OLS(Y,X_1)
result = model.fit()
nw_results = result.get_robustcov_results(cov_type='HAC', maxlags=nw_lags)

nw_results.params

array([129.16437329,  44.63372485, -12.61389722,  -3.6435151 ,
        42.8580515 , -59.17368386, -52.55581668,  -2.31210659,
        -3.726146  ,  -7.29710135, -28.60422653, 159.2707804 ,
        45.14554603,  18.81016165,  54.63133684, -11.79241125,
       -66.62038633])

In [27]:
nw_results.summary(xname = ['const'] + regressor_columns_def)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.644
Method:                 Least Squares   F-statistic:                     403.1
Date:                Sun, 30 Nov 2025   Prob (F-statistic):               0.00
Time:                        15:05:10   Log-Likelihood:            -1.0606e+05
No. Observations:               17379   AIC:                         2.122e+05
Df Residuals:                   17362   BIC:                         2.123e+05
Df Model:                          16                                         
Covariance Type:                  HAC                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          129.1644      4.367     29.579      0.000     120.605     137.724
temp            44.6337      2.659     16.788      0.000      39.423      49.845
hum            -12.6139      1.709     -7.381      0.000     -15.963      -9.264
windspeed       -3.6435      1.291     -2.822      0.005      -6.174      -1.113
yr              42.8581      1.240     34.555      0.000      40.427      45.289
hr_sin         -59.1737      1.581    -37.427      0.000     -62.273     -56.075
hr_cos         -52.5558      1.498    -35.083      0.000     -55.492     -49.620
weekday_sin     -2.3121      1.204     -1.920      0.055      -4.672       0.048
day_cos         -3.7261      1.137     -3.278      0.001      -5.954      -1.498
mnth_sin        -7.2971      2.986     -2.444      0.015     -13.150      -1.444
holiday        -28.6042      9.417     -3.037      0.002     -47.063     -10.146
rush_hour      159.2708      5.729     27.801      0.000     148.041     170.500
season_2        45.1455      4.473     10.093      0.000      36.379      53.913
season_3        18.8102      7.084      2.655      0.008       4.924      32.696
season_4        54.6313      5.837      9.360      0.000      43.191      66.072
weathersit_2   -11.7924      2.553     -4.619      0.000     -16.797      -6.788
weathersit_3   -66.6204      5.370    -12.405      0.000     -77.147     -56.094
==============================================================================
Omnibus:                     2110.243   Durbin-Watson:                   0.728
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3994.669
Skew:                           0.790   Prob(JB):                         0.00
Kurtosis:                       4.738   Cond. No.                         10.5
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 24 lags and without small sample correction
"""

---
Tras la obtención de nuestro modelo, procedemos a guardarlo para uso futuro.

In [28]:
X_1.dump('../Data/X_def.dat')
nw_results.params.dump('../Data/B_def.dat')
Y.dump('../Data/Y_def.dat')

In [29]:
nw_results.save("../Data/ols_def_results.pickle")

In [30]:
with open("../Data/regressor_columns_def.pickle", "wb") as fp:   #Pickling
...   pickle.dump(regressor_columns_def, fp)